In [1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import datetime
import string
from string import digits
import collections
import scipy.stats as scs
import cc_pipeline as P
import time
import random

#sentiment and language
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import vaderSentiment
from langdetect import detect
from gensim.models import Word2Vec
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF, LatentDirichletAllocation
import knee_locator




#plotting
from bokeh.plotting import figure, show, output_notebook
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:

def scrape_text(series):
    
    #use pd.series instead moving forward
    text = []
    for link in series:
        
        try:
            req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
            webpage = webpage.decode('utf-8')
            text.append(webpage)
        except SocketError as e:
            if e.errno != errno.ECONNRESET:
                raise # Not error we are looking for
            pass # Handle error here.
        
    return text

def scrape_from_url(cycles):
    
    for i in range(cycles):
        
        time.sleep(900)
        
        from_df = pd.read_csv('data/cc_recent.csv', encoding='utf-8')
        temp_df = from_df.head(1000)

        #scrape text then concat
        text_url_series = pd.Series(temp_df['url'])
        extracts = scrape_text(text_url_series)
        extracts = pd.Series(extracts)
        temp_df['text'] = extracts

        head_df = pd.read_csv('data/cc_head_text.csv', encoding='utf-8')
        head_df = pd.concat([head_df, temp_df])
        head_df.to_csv('data/cc_head_text.csv', encoding='utf-8', index=False)

        from_df = from_df.drop(from_df.index[0:1000])
        from_df.to_csv('data/cc_recent.csv', encoding='utf-8', index=False)

        head_df=None
        temp_df=None
        from_df=None
        text_url_series=None
        extracts=None
        


In [3]:
#scrape_from_url(1)

In [4]:
#clean text

def get_show_text(show_raw):
    
    '''returns show text without timestamps'''
    
    return "\n".join( ["\n".join( x.split("\n")[2:] ) for x in show_raw.split("\n\n")] )



def clean_all_text(text_list):
    
    '''cleans all text and creates new column in dataframe'''
    
    doc_l
    for word in text_list:
        doc_list.append(get_show_text(word))
    return doc_list


def clean_text(doc):
    '''cleans and lemmatizes a string by removing punc, characters, digits, and len(words) < 3'''
    
    stop_words = stopwords.words('english')
    punct = ('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~♪¿’')
    remove_digits = str.maketrans('', '', digits)
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized = []
    
    doc = doc.split('\n')
    doc = ' '.join(doc)
    doc = doc.split('-')
    doc = ' '.join(doc)
    doc = doc.split('...')
    doc = ' '.join(doc)
    doc = word_tokenize(doc)

    a = [char for char in doc if char not in punct]
    b = [w for w in a if w not in stop_words] 
    c = [w for w in b if len(w) > 3]
    d = [x for x in c if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]

    e = ' '.join(d)
    f = e.lower()
    g = f.translate(remove_digits)
    cleaned = str(g)
    doc = word_tokenize(cleaned)
    
    for val in doc:
        doc_temp = wordnet_lemmatizer.lemmatize(val)
        lemmatized.append(doc_temp)
    doc = ' '.join(lemmatized)
    
    return doc

def clean_and_return(docs_list):
    
    docs = []
    for cc in docs_list:
        cleaned_temp = clean_text(cc)
        docs.append(cleaned_temp)
        
    return docs


def lang_detect(doc_series):
    
    lang = []
    for x in doc_series:
        eng = 'en'
        span = 'es'

        try:
            if detect(x) == eng:
                lang.append(eng)
            else:
                lang.append(span)
        except:
            lang.append(None)
            
    return lang

In [5]:
temp_df = pd.read_csv('data/cc_1000_text.csv', encoding='utf-8')
temp_df.head()


,_id,channel,created_at,duration,lang,machine_id,ts,tui,tv,url,zip_url,date,text
0,{'$oid': '5bb2b5082c1ba60007fc013d'},WBNX_HD_CW,{'$date': '2018-10-02T00:00:08.982Z'},900000,dflt,ENSWERCCR7_8_3,{'$numberLong': '1538437500000'},251536767,{'$date': '2018-10-01T23:45:00.000Z'},https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,2018-10-02 00:00:08.982,1538437621406\n1835\n[ KNOCK ON DOOR ]\n\n1538...
1,{'$oid': '5bb2b5106f40ed000728b879'},KZJO_MY_NETWORK_SEATTLE,{'$date': '2018-10-02T00:00:16.702Z'},900000,dflt,ENSWERCCR7_48_2,{'$numberLong': '1538437500000'},251533435,{'$date': '2018-10-01T23:45:00.000Z'},https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,2018-10-02 00:00:16.702,"1538437521225\n1435\nSeen it, covered it.\n\n1..."
2,{'$oid': '5bb2b51170969a000bc9a14e'},WVUA_THIS_TV_BIRMINGHAM,{'$date': '2018-10-02T00:00:17.995Z'},900000,dflt,ENSWERCCR7_65_236,{'$numberLong': '1538435700000'},251541606,{'$date': '2018-10-01T23:15:00.000Z'},https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,2018-10-02 00:00:17.995,1538435707355\n4571\n\n\n\n1538435711926\n23...
3,{'$oid': '5bb2b51187701400079ec41d'},TV_LAND,{'$date': '2018-10-02T00:00:17.925Z'},900000,dflt,ENSWERCCR4_22,{'$numberLong': '1538437500000'},251536903,{'$date': '2018-10-01T23:45:00.000Z'},https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,2018-10-02 00:00:17.925,1538437500869\n2653\nWe’ll talk later.\n\n1538...
4,{'$oid': '5bb2b511b18af90007b4e445'},WPCH_INDEPENDENT_ATLANTA,{'$date': '2018-10-02T00:00:17.074Z'},900000,dflt,ENSWERCCR7_7_245,{'$numberLong': '1538437500000'},251534109,{'$date': '2018-10-01T23:45:00.000Z'},https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,https://s3.amazonaws.com/adm-mpeg-dash/gnvideo...,2018-10-02 00:00:17.074,1538437500144\n1201\nIt hurts.\n\n153843750218...


In [6]:

temp = temp_df['text'].values
temp = temp.tolist()
type(temp)



list

In [8]:
docs_list = clean_all_text(temp)
type(docs_list)




NameError: name 'doc_list' is not defined

In [ ]:
cleaned_list = clean_and_return(docs_list)


In [ ]:
temp_df['cleaned'] = cleaned_list
temp_df.head()

In [ ]:
doc_series = pd.Series(temp_df['cleaned'].values)
language = lang_detect(doc_series)
temp_df['language'] = language
english = temp_df[temp_df['language'] == 'en']
spanish = temp_df[temp_df['language'] == 'es']



In [ ]:
# english.to_csv('data/testenglish.csv', encoding='utf-8', index=False)
# spanish.to_csv('data/testspanish.csv', encoding='utf-8', index=False)
english.head()